In [1]:
from typing import List
import numpy as np
import logging

import Bio
import Bio.PDB
import Bio.SVDSuperimposer

In [6]:
def extract_variable_regions_residues(
    structure: Bio.PDB.Structure.Structure, 
    hchain_id: str, 
    lchain_id: str):
    '''
    Extracts residues of the variable regions. It's used to extract
     a constant length for heavy and light chains, as to allow direct
     structural alignments (e.g. based on Calpha carbons).
    '''

    H_CHAIN_LEN = 113 + 12
    L_CHAIN_LEN = 107 + 11

    if len(structure) > 1:
        logging.warning(f"Structure {structure.id} has >1 model")
    model = structure[0]
    hchain = model[hchain_id]
    lchain = model[lchain_id]

    hchain_res = list(hchain.get_residues())[:H_CHAIN_LEN]
    lchain_res = list(lchain.get_residues())[:L_CHAIN_LEN]

    return hchain_res, lchain_res

def is_atom_ca(atom: Bio.PDB.Atom.Atom) -> bool:
    return atom.get_id() == "CA"

def get_atom_coord(atom: Bio.PDB.Atom.Atom) -> np.array:
    '''Returns list with 3 elements ~ coordinates - vector.'''
    return np.array(atom.get_coord())

def extract_calpha_coord(residues: List[Bio.PDB.Residue.Residue]) -> np.array:
    coord_array = None
    for r in residues:
        ca_found = False
        atoms = r.get_atoms()        
        for atom in atoms:
            if is_atom_ca(atom):
                if ca_found:
                    raise RuntimeError("Found >1 Calpha carbons for a residue.")
                coord = get_atom_coord(atom)
                ca_found = True
        
        if coord_array is not None:
            coord_array = np.vstack([coord_array, coord])
        else:
            coord_array = coord
    return coord_array

def compute_rmsd(
    coord_1: np.array, 
    coord_2: np.array, 
    superimposer = None) -> float:
    
    if superimposer is None:
        superimposer = Bio.SVDSuperimposer.SVDSuperimposer()
    superimposer.set(coord_1, coord_2)
    superimposer.run()
    return superimposer.get_rms()


4.655095074888375 6.572240819227812


In [12]:
parser = Bio.PDB.PDBParser(PERMISSIVE=0)  # strict parser
structure1 = parser.get_structure("test1", "data/SAbDab/chothia/1ad0.pdb")
structure2 = parser.get_structure("test2", "data/SAbDab/chothia/1fgv.pdb")

h_res_1, l_res_1 = extract_variable_regions_residues(structure1, "B", "A")
h_res_2, l_res_2 = extract_variable_regions_residues(structure2, "H", "L")

coord_h1 = extract_calpha_coord(h_res_1)
coord_l1 = extract_calpha_coord(l_res_1)
coord_h2 = extract_calpha_coord(h_res_2)
coord_l2 = extract_calpha_coord(l_res_2)

sup = Bio.SVDSuperimposer.SVDSuperimposer()
rmsd_h12 = compute_rmsd(coord_h1, coord_h2, sup)
rmsd_l12 = compute_rmsd(coord_l1, coord_l2, sup)
rmsd_v12 = compute_rmsd(
    np.vstack([coord_h2, coord_l2]),
    np.vstack([coord_h1, coord_l1]),
)

print(rmsd_h12, rmsd_l12)

4.655095074888375 6.572240819227812 5.800130139316306


In [ ]:
# Sample pairs and compute rmsd